## Scraping data from a public website

This notebook tracks my progress in scraping the website [snlarchives](http://www.snlarchives.net) for a Saturday Night Live dataset. I chose to use the python library [scrapy](https://scrapy.org/) to do the work.

### Imports

In [1]:
import scrapy
import pandas as pd
import numpy as np

### Scrapy spiders

Scrapy uses so called spiders to crawl the web. This means that I had to create a subclass from scrapy.Spider to begin. Spider *spider123* can be executed by using 

    scrapy runspider spider123.py 

in the python command line. This means that the code for each spider needs to be saved into a .py-file.

In [2]:
class snl(scrapy.Spider):
    name = 'snl'
    start_urls = ['http://www.snlarchives.net/Seasons/']
    base_url = "http://www.snlarchives.net"

    actor_seen = set()

    def parse(self, response):

        snl = {}

        # parsing snlarchives. Entrypoint is the seasons page at www.snlarchives.net/Seasons/
        for season in response.css('div.thumbRectInner'):
            sid = int(season.css('::text').extract_first())
            year = 1974 + sid
            next_page = '?{}'.format(year)

            item_season = {}
            item_season['sid'] = sid
            item_season['year'] = year
            item_season['type'] = 'season'

            yield item_season

            yield scrapy.Request(response.urljoin(next_page), callback=self.parseSeason, meta={'season': item_season})
            # remove statement to scrape more than one season
            #break
            
    def parseSeason(self, response):
        # parsing a season (e.g. www.snlarchives.net/Seasons/?1975)
        # episodes is already chosen
        item_season = response.meta['season']

        for episode in response.css('a'):
            href_url = episode.css("a ::attr(href)").extract_first()
            if href_url.startswith("/Episodes/?") and len(href_url)==19: 
                episode_url = self.base_url + href_url
                yield scrapy.Request(episode_url, callback=self.parseEpisode, meta={'season': item_season, 'date': href_url.split('?')[1]})
                # remove statement to scrape more than one episode
                #break

    def parseEpisode(self, response):
        item_season = response.meta['season']

        episode = {}
        episode['date'] = response.meta['date']
        episode['sid'] = item_season['sid']
        episode['year'] = item_season['year']
        episode['type'] = 'episode'

        for epInfoTr in response.css("table.epGuests tr"):
            epInfoTd = epInfoTr.css("td")
            if epInfoTd[0].css("td p ::text").extract_first() == 'Aired:':
                episode['aired'] = epInfoTd[1].css("td p ::text").extract()
                break

        yield episode
        # initially the titles tab is opened
        for sketchInfo in response.css("div.sketchWrapper"):
            sketch = {}
            href_url = sketchInfo.css("a ::attr(href)").extract_first()
            sketch['sid'] = item_season['sid']
            sketch['tid'] = href_url.split('?')[1]
            sketch['date'] = episode['date']
            sketch['title'] = sketchInfo.css(".title ::text").extract_first()
            sketch['type'] = 'title'
            sketch['titleType'] = sketchInfo.css(".type ::text").extract_first()

            sketch_url = self.base_url + href_url
            yield scrapy.Request(sketch_url, callback=self.parseTitle, meta={'title': sketch})
            # remove statement to scrape more than one sketch
            #break

    def parseTitle(self, response):
        sketch = response.meta['title']
        for actor in response.css(".roleTable > tr"):
            actor_dict = {}
            actor_sketch = {}
            actor_dict['name'] = actor.css("td ::text").extract_first()
            actor_dict['type'] = 'actor'
            href_url = actor.css("td > a ::attr(href)").extract_first()
            if href_url != None and href_url.split('?')[0] == '/Cast/':
                actor_dict['aid'] = href_url.split('?')[1]
                actor_dict['isCast'] = True
                actor_sketch['actorType'] = 'cast'
                # possible sraping of the cast members here
            else:
                actor_dict['aid'] = actor_dict['name']
                actor_dict['isCast'] = False
                actor_sketch['actorType'] = actor.css("td ::attr(class)").extract_first()
            
            if not actor_dict['aid'] in self.actor_seen:
                self.actor_seen.add(actor_dict['aid'])
                yield actor_dict

            actor_sketch['tid'] = sketch['tid']
            actor_sketch['sid'] = sketch['sid']
            actor_sketch['aid'] = actor_dict['aid']
            actor_sketch['type'] = 'actor_sketch'
            yield actor_sketch
        yield sketch

### Running the spider
You can also run a spider from a script. To do this you need to import the additional class *CrawlerProcess*. Note that scraping can take a lot of time, so I inserted *break*-statements into *snl_spider*. The code above will only scrape one sketch of one episode of one season. This will be done in a reasonable time. If you want to scrape everything you should call your spider from the commandline using the command above.

In [3]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'FEED_FORMAT': 'json',
    'FEED_URI': 'snl.json'
})

#process.crawl(snl_spider)
#process.start()

2017-02-08 08:14:17 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-02-08 08:14:17 [scrapy] INFO: Overridden settings: {'FEED_URI': 'snl.json', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)', 'FEED_FORMAT': 'json'}


### Looking at the data
Once the spider is finished you have a .json-file that contains all of your data. Let's read it into a pandas dataframe.

In [4]:
snl_df = pd.read_json("./data/snl_v9.json")

Now let's look at the columns of the dataframe.

In [5]:
snl_df.columns

Index(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'Aged 18-29',
       'Aged 18-29_avg', 'Aged 30-44', 'Aged 30-44_avg', 'Aged 45+',
       'Aged 45+_avg', 'Aged under 18', 'Aged under 18_avg', 'Females',
       'Females Aged 18-29', 'Females Aged 18-29_avg', 'Females Aged 30-44',
       'Females Aged 30-44_avg', 'Females Aged 45+', 'Females Aged 45+_avg',
       'Females under 18', 'Females under 18_avg', 'Females_avg', 'IMDb staff',
       'IMDb staff_avg', 'IMDb users', 'IMDb users_avg', 'Males',
       'Males Aged 18-29', 'Males Aged 18-29_avg', 'Males Aged 30-44',
       'Males Aged 30-44_avg', 'Males Aged 45+', 'Males Aged 45+_avg',
       'Males under 18', 'Males under 18_avg', 'Males_avg', 'Non-US users',
       'Non-US users_avg', 'Top 1000 voters', 'Top 1000 voters_avg',
       'US users', 'US users_avg', 'actorType', 'aid', 'aired', 'eid',
       'isCast', 'name', 'sid', 'tid', 'title', 'titleType', 'type', 'year'],
      dtype='object')

As you can see the data frame contains every column from every type of data. This means that the first thing to do is split the data into separate dataframes. To do this I inserted the column *type* to every entry.

In [73]:
snl_season = snl_df[snl_df.type == 'season'][['sid','year']].reset_index()
snl_episode = snl_df[snl_df.type == 'episode'][['sid','eid','year','aired']].reset_index()
snl_title = snl_df[snl_df.type == 'title'][['sid','eid','tid','title','titleType']].reset_index()
snl_actor = snl_df[snl_df.type == 'actor'][['aid','name','isCast']].reset_index()
snl_actor_sketch = snl_df[snl_df.type == 'actor_sketch'][['sid','eid','tid','aid','actorType']].reset_index()
snl_rating = snl_df[snl_df.type == 'rating'][['sid','eid','1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'Aged 18-29',
       'Aged 18-29_avg', 'Aged 30-44', 'Aged 30-44_avg', 'Aged 45+',
       'Aged 45+_avg', 'Aged under 18', 'Aged under 18_avg', 'Females',
       'Females Aged 18-29', 'Females Aged 18-29_avg', 'Females Aged 30-44',
       'Females Aged 30-44_avg', 'Females Aged 45+', 'Females Aged 45+_avg',
       'Females under 18', 'Females under 18_avg', 'Females_avg', 'IMDb staff',
       'IMDb staff_avg', 'IMDb users', 'IMDb users_avg', 'Males',
       'Males Aged 18-29', 'Males Aged 18-29_avg', 'Males Aged 30-44',
       'Males Aged 30-44_avg', 'Males Aged 45+', 'Males Aged 45+_avg',
       'Males under 18', 'Males under 18_avg', 'Males_avg', 'Non-US users',
       'Non-US users_avg', 'Top 1000 voters', 'Top 1000 voters_avg',
       'US users', 'US users_avg']].reset_index()

Fix some datatypes

In [74]:
snl_rating['sid'] = snl_rating['sid'].astype(int)
snl_rating['eid'] = snl_rating['eid'].astype(int)
snl_rating['1'] = snl_rating['1'].fillna(0).astype(int)
snl_rating['2'] = snl_rating['2'].fillna(0).astype(int)
snl_rating['3'] = snl_rating['3'].fillna(0).astype(int)
snl_rating['4'] = snl_rating['4'].fillna(0).astype(int)
snl_rating['5'] = snl_rating['5'].fillna(0).astype(int)
snl_rating['6'] = snl_rating['6'].fillna(0).astype(int)
snl_rating['7'] = snl_rating['7'].fillna(0).astype(int)
snl_rating['8'] = snl_rating['8'].fillna(0).astype(int)
snl_rating['9'] = snl_rating['9'].fillna(0).astype(int)
snl_rating['10'] = snl_rating['10'].fillna(0).astype(int)
snl_rating['Aged under 18'] = snl_rating['Aged under 18'].fillna(0).astype(int)
snl_rating['Aged 18-29'] = snl_rating['Aged 18-29'].fillna(0).astype(int)
snl_rating['Aged 30-44'] = snl_rating['Aged 30-44'].fillna(0).astype(int)
snl_rating['Aged 45+'] = snl_rating['Aged 45+'].fillna(0).astype(int)
snl_rating['Females under 18'] = snl_rating['Females under 18'].fillna(0).astype(int)
snl_rating['Females Aged 18-29'] = snl_rating['Females Aged 18-29'].fillna(0).astype(int)
snl_rating['Females Aged 30-44'] = snl_rating['Females Aged 30-44'].fillna(0).astype(int)
snl_rating['Females Aged 45+'] = snl_rating['Females Aged 45+'].fillna(0).astype(int)
snl_rating['Males under 18'] = snl_rating['Males under 18'].fillna(0).astype(int)
snl_rating['Males Aged 18-29'] = snl_rating['Males Aged 18-29'].fillna(0).astype(int)
snl_rating['Males Aged 30-44'] = snl_rating['Males Aged 30-44'].fillna(0).astype(int)
snl_rating['Males Aged 45+'] = snl_rating['Males Aged 45+'].fillna(0).astype(int)
snl_rating['IMDb staff'] = snl_rating['IMDb staff'].fillna(0).astype(int)
snl_rating['IMDb users'] = snl_rating['IMDb users'].fillna(0).astype(int)
snl_rating['US users'] = snl_rating['US users'].fillna(0).astype(int)
snl_rating['Non-US users'] = snl_rating['Non-US users'].fillna(0).astype(int)
snl_rating['Top 1000 voters'] = snl_rating['Top 1000 voters'].fillna(0).astype(int)

In [65]:
snl_episode = snl_episode[np.isfinite(snl_episode['eid'])]

In [71]:
snl_season.sid = pd.to_numeric(snl_season.sid)
snl_season.year = pd.to_numeric(snl_season.year)

snl_episode.sid = pd.to_numeric(snl_episode.sid)
snl_episode.eid = pd.to_numeric(snl_episode.eid)
snl_episode.year = pd.to_numeric(snl_episode.year)

snl_title.sid = pd.to_numeric(snl_title.sid)
snl_title.eid = pd.to_numeric(snl_title.eid)
snl_title.tid = pd.to_numeric(snl_title.tid)

snl_actor.isCast = pd.to_numeric(snl_actor.isCast)

snl_actor_sketch.sid = pd.to_numeric(snl_actor_sketch.sid)
snl_actor_sketch.eid = pd.to_numeric(snl_actor_sketch.eid)
snl_actor_sketch.tid = pd.to_numeric(snl_actor_sketch.tid)

Let's look at the first 10 sketches in our database.

In [72]:
snl_title.head(10)

,sid,eid,tid,title,titleType
82,2,12,197701221,Injured John,Cold Opening
91,3,20,197805201,Nixon's Book,Cold Opening
99,3,19,197805131,Paraquat,Cold Opening
113,6,8,198101241,America Not Held Hostage Anymore,Cold Opening
118,6,13,198104111,Storage Room,Cold Opening
121,7,13,198202271,CBS Evening News,Cold Opening
135,4,20,197905261,Mr. Bill,Cold Opening
151,3,18,197804221,Rock Concert,Cold Opening
153,7,19,198205151,Kaufman Fight,Cold Opening
158,7,20,198205221,Women's Room,Cold Opening


Now let's store the files on the disc so that we can share them.

In [23]:
snl_season.to_csv("./db/snl_season.csv")
snl_episode.to_csv("./db/snl_episode.csv")
snl_title.to_csv("./db/snl_title.csv")
snl_actor.to_csv("./db/snl_actor.csv")
snl_actor_sketch.to_csv("./db/snl_actor_sketch.csv")
snl_ratings.to_csv("./db/snl_ratings.csv")